<a href='https://www.darshan.ac.in/'> <img src='https://www.darshan.ac.in/Content/media/DU_Logo.svg' width="250" height="300"/></a>
<pre>
<center><b><h1>Data Mining</b></center>
<center><b><h1>Lab - 7 (Part 2)</b></center>    
<pre>

### Step 1: Load the Dataset
Load the `Tdata.csv` file and display the first few rows.

In [1]:
import pandas as pd
df = pd.read_csv("Tdata.csv")
df.head()

Transaction  bread  butter  coffee  eggs  jam  milk
0          T1      1       1       0     0    0     1
1          T2      1       1       0     0    1     0
2          T3      1       0       0     1    0     1
3          T4      1       1       0     0    0     1
4          T5      1       0       1     0    0     0

### Step 2: Drop the 'Transaction' Column
We're only interested in the items (not the transaction IDs).

In [2]:
df_items = df.drop(columns=['Transaction'])
df_items.head()

bread  butter  coffee  eggs  jam  milk
0      1       1       0     0    0     1
1      1       1       0     0    1     0
2      1       0       0     1    0     1
3      1       1       0     0    0     1
4      1       0       1     0    0     0

### Step 3: Count Single Items
See how many transactions include each item.

In [3]:
df_items.sum()

bread     5
butter    3
coffee    2
eggs      2
jam       2
milk      3
dtype: int64

### Step 4: Define Apriori Function
This function finds frequent itemsets of size 1, 2, and 3 with minimum support.

In [18]:
from itertools import combinations

def find_frequent_itemsets(df, min_support):
    n = len(df)
    result = []

    for k in [1, 2, 3]:  # for 1-item, 2-item, 3-item sets
        for items in combinations(df.columns, k):
            mask = df[list(items)].all(axis=1)
            support = mask.sum() / n
            if support >= min_support:
                result.append((frozenset(items), round(support, 2)))

    return result

### Step 5: Run Apriori
Set `min_support = 0.6` and display the frequent itemsets.

In [21]:
frequent_itemsets = find_frequent_itemsets(df_items, min_support=0.5)


for itemset, support in frequent_itemsets:
    print(f"{set(itemset)} → support: {support}")

{'bread'} → support: 0.83
{'butter'} → support: 0.5
{'milk'} → support: 0.5
{'butter', 'bread'} → support: 0.5
{'milk', 'bread'} → support: 0.5


### Step 6 Display as a DataFrame

In [13]:

result_df = pd.DataFrame(frequent_itemsets, columns=['Itemset', 'Support'])
result_df

Itemset  Support
0          (bread)     0.83
1         (butter)     0.50
2           (milk)     0.50
3  (butter, bread)     0.50
4    (milk, bread)     0.50

# Orange Tool : - >Generate Same Frequent Patterns in Orange tools

# Extra : - >  Define Apriori Function without itertools

In [11]:
def find_frequent_itemsets(df, min_support):
    n = len(df)
    result = []

    # Step 1: Frequent 1-itemsets (L1)
    L = []
    for col in df.columns:
        support = df[col].sum() / n
        if support >= min_support:
            itemset = frozenset([col])
            L.append(itemset)
            result.append((itemset, round(support, 2)))

    k = 2
    while L and k <= 3:  # Step 2: Only generate up to 3-itemsets
        Ck = []
        for i in range(len(L)):
            for j in range(i + 1, len(L)):
                union = L[i] | L[j]
                if len(union) == k and union not in Ck:
                    # Prune: all (k-1)-subsets must be frequent
                    subsets = [union - {item} for item in union]
                    if all(sub in L for sub in subsets):
                        Ck.append(union)

        # Step for Support Count 
        Lk = []
        for cand in Ck:
            cols = list(cand)
            support = df[cols].all(axis=1).sum() / n
            if support >= min_support:
                Lk.append(cand)
                result.append((cand, round(support, 2)))

        L = Lk
        k += 1

    return result
